Импорт библиотек

In [649]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Всего датасет содержит 159 записей. Мы предварительно зарезервировали часть датасета для задания по преодолению бейслайна (baseline). В настоящем задании подразумевается работа с датасетом, содержащим 119 записей.

# 0. Применение полиномиальной регрессии для предсказания непрерывного параметра

Представленный набор данных — это набор данных о семи различных типах рыб, продаваемых в некоторой рыбной лавке. Наша цель заключается в том, чтобы предсказать массу рыбы по каким-то косвенным признакам, известным о рыбе. Сами признаки, быть может, нужно синтезировать из тех, что известны.

При помощи <code>train_test_split()</code> разбейте набор данных на обучающую и тестовую выборки с параметрами, указанными в вашем задании. Используйте стратификацию по колонке <code>Species</code>. Стратификация позволит сохранить доли представленных объектов (по представителям типов рыб) в тренировочной и тестовой выборках.

In [650]:
rew=pd.read_csv('/kaggle/input/fish-train/fish_train.csv')
#rew_res = pd.read_csv('/kaggle/input/fish-train/fish_resered.csv')


train_x,test_x,train_y,test_y = train_test_split(rew.drop(['Weight'],axis=1), rew['Weight'],test_size=0.2, random_state=23,stratify=rew['Species'])
rew.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Roach,150.0,20.4,22.0,24.7,5.8045,3.7544
1,Perch,250.0,25.4,27.5,28.9,7.2828,4.5662
2,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340
3,Bream,925.0,36.2,39.5,45.3,18.7542,6.7497
4,Perch,110.0,19.0,21.0,22.5,5.6925,3.5550


Вычислите выборочное среднее колонки <code>Width</code> полученной тренировочной выборки.

In [651]:

train_x['Width'].mean()
test_x.shape

(24, 6)

# 1. Построение базовой модели

Избавьтесь от категориальных признаков и обучите модель линейной регрессии (<code>LinearRegression()</code>) на тренировочном наборе данных. Выполните предсказания для тестового набора данных. Оцените модель при помощи метрики <code>r2_score()</code>.

In [652]:
train_x_copy = train_x
test_x_copy = test_x

train_x = train_x.drop(['Species'], axis=1)
test_x = test_x.drop(['Species'], axis=1)


model = LinearRegression()
model.fit(train_x, train_y)

y_pred = model.predict(test_x)
r2_score1 = r2_score(test_y, y_pred)
print(r2_score1)

##reg = LinearRegression().fit(train_x, train_y)
#y_pred = reg.predict(test_x)
#r2_score = r2_score(test_y, y_pred)
#test_y.head()

0.8885408749541969


# 2. Добавление предварительной обработки признаков.

## Использование PCA

При помощи, например, <code>sns.heatmap()</code>, постройте матрицу корреляций признаков тренировочного набора данных и определите тройку наиболее коррелированных между собой признаков.

In [653]:
# < ENTER YOUR CODE HERE > 

Линейные модели достаточно плохо реагируют на коррелированные признаки, поэтому от таких признаков имеет смысл избавиться еще до начала обучения.

Для уменьшения количества неинформативных измерений используйте метод главных компонент. 

1) Примените метод главных компонент (<code>PCA(n_components=3, svd_solver='full')</code>) для трех найденных наиболее коррелированных признаков. 

2) Вычислите долю объясненной дисперсии при использовании только первой главной компоненты. 

3) Замените три наиболее коррелированных признака на новый признак <code>Lengths</code>, значения которого совпадают со значениями счетов первой главной компоненты.

In [654]:
# < ENTER YOUR CODE HERE > 

Примените полученное преобразование для тех же признаков в тестовом наборе данных. Обратите внимание, что заново обучать преобразование `PCA` не нужно. Аналогично предыдущему этапу замените три рассмотренных признака на один.

In [655]:
train_pca = train_x[['Length1','Length2','Length3']]
test_pca = test_x[['Length1','Length2','Length3']]
pca = PCA(n_components=1)

train_num_x_tr = pca.fit_transform(train_pca)
test_num_x_tr = pca.transform(test_pca)

train_x['Lengths'] = train_num_x_tr
test_x['Lengths'] = test_num_x_tr

test_x.head()

train_x = train_x.drop(['Length1','Length2','Length3'],axis=1)
test_x = test_x.drop(['Length1','Length2','Length3'],axis=1)

#train_y = train_y.to_numpy()
#test_y = test_y.to_numpy()

Обучите базовую модель линейной регресси на полученных тренировочных данных, снова выбросив категориальные признаки. Выполните предсказания для тестовых данных, оцените при помощи <code>r2_score()</code>.

In [656]:
model2 = LinearRegression()
model2.fit(train_x, train_y)

y_pred = model2.predict(test_x)
r2_score2 = r2_score(test_y, y_pred)
print(r2_score2)

0.8925481416058358


Видно, что точность значительно не изменилась.

## Модификация признаков

Постройте графики зависимостей признаков от целевой переменной, например, при помощи <code>sns.pairplot()</code>.

In [657]:
for c in train_x.columns:
    train_x[c] = train_x[c]**3
for c in test_x.columns:
    test_x[c] = test_x[c]**3
train_x['Width'].mean()

124.41840117423388

Видно, что масса, вообще говоря, нелинейно зависит от остальных параметров. Значит, чтобы линейная модель хорошо справлялась с предсказанием, признаки имеет смысл преобразовать так, чтобы зависимость стала более похожей на линейную. Но как придумать такую зависимость?

Логично предположить, что масса рыбы должна каким-то гладким образом зависеть от остальных параметров, отвечающих так или иначе за размеры. Если впомнить, что масса — это произведение плотности на объем, то

$$
m = \rho \cdot V.
$$

Допустим, что средняя плотность у всех рыб одинаковая, и вспомним, что при гомотетии объем объекта зависит от линейных размеров как куб, тогда получим

$$
m\sim V\sim d^3
$$

Все признаки тренировочного и тестового наборов данных, отвечающие так или иначе за размеры (<code>Height, Width, Lengths</code>), возведите в третью степень, и проверьте, стала ли зависимость массы от этих признаков похожа на линейную.

In [658]:
# < ENTER YOUR CODE HERE > 

Введите выборочное среднее колонки <code>Width</code> тренировочного набора данных после возведения в куб.

In [659]:
# < ENTER YOUR CODE HERE > 

Выберите изображения, соответствующие зависимости <code>Weight</code> от <code>Width</code> до преобразования и после.

Обучите базовую модель линейной регресси на полученных тренировочных данных, снова выбросив категориальные признаки. Выполните предсказания для тестовых данных, оцените при помощи `r2_score()`.

In [660]:
model3 = LinearRegression()
model3.fit(train_x, train_y)

y_pred3 = model3.predict(test_x)
r2_score3 = r2_score(test_y, y_pred3)
print(r2_score3)

0.9452890309259311


Обратите внимание на то, как такая нехитрая работа с признаками помогла разительно улучшить точность модели!

## Добавление категориальных признаков

Произведите <code>one-hot</code> кодировние категориального признака `Species`, например, с помощью <code>pd.get_dummies()</code>.

Обучите модель линейной регресси на полученных тренировочных данных. Выполните предсказания для тестовых данных, оцените модель при помощи <code>r2_score()</code>.

<b>Примечание</b>: Мы специально использовали стратифицированное разделение, чтобы все значения категориального признака <code>Species</code> присутствовали во всех наборах данных. Но такое возможно не всегда. Про то, как с этим бороться можно почитать, [например, здесь](https://predictivehacks.com/?all-tips=how-to-deal-with-get_dummies-in-train-and-test-dataset).

In [661]:
train_x=train_x.join(rew['Species'])
test_x=test_x.join(rew['Species'])

train_x_dum = pd.get_dummies(train_x, columns=['Species'],dtype=int)
test_x_dum = pd.get_dummies(test_x, columns=['Species'],dtype=int)

model4 = LinearRegression()
model4.fit(train_x_dum, train_y)

y_pred4 = model4.predict(test_x_dum)
r2_score4 = r2_score(test_y, y_pred4)
print(r2_score4)

0.9525713724639514


И снова точность возрасла.

Как можно увидеть, после `one-hot` кодирования признаки стали коррелированы. От этого можно избавиться, например, при помощи параметра `drop_first=True`. Заново обучите модель после исправления этого недочета. Выполните предсказания для тестовых данных, оцените модель при помощи <code>r2_score()</code>.

In [662]:
# < ENTER YOUR CODE HERE > 

На таком сравнительно небольшом наборе данных, впрочем, разницы мы не видим.